## API Test Document

#### Setup
The next cell includes the setup (in Python). If you want to run the code, you will have to install supabase:
`pip3 install supabase`
You will also need to create a file, "keys.py", put it into the `data` folder, and paste in the API key provided to you. 
NEVER commit the key file. 

[Supabase Documentation](https://supabase.com/docs/reference/python/introduction)

In [ ]:
# imports
from transformers import AutoTokenizer, AutoModel
import torch
import os

# Initialize ClinicalBERT model
local_model_path = "/Users/zrb/Downloads/Technation-Healthsync-2025-main/local_model"

try:
    # 尝试使用本地模型
    print("正在加载本地ClinicalBERT模型...")
    tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")  # 使用预训练tokenizer
    model = AutoModel.from_pretrained(local_model_path)
    print("本地ClinicalBERT模型加载完成")
except Exception as e:
    print(f"本地模型加载失败: {e}")
    print("回退到在线模型...")
    tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
    model = AutoModel.from_pretrained("medicalai/ClinicalBERT")
    print("在线ClinicalBERT模型加载完成")

# Supabase setup (optional - only if you have a valid API key)
try:
    from supabase import create_client, Client
    
    url: str = "https://sthnvivctugihwhomsxu.supabase.co" # the URL endpoint
    
    # Try to get the Supabase key from environment variable first, then from keys file
    try:
        key: str = os.environ.get("SUPABASE_KEY")
        if not key:
            import data.keys as keys
            key: str = keys.SUPABASE_KEY
    except ImportError:
        print("Warning: data/keys.py not found. Please set SUPABASE_KEY environment variable or create data/keys.py")
        key = "your_supabase_key_here"  # Placeholder
    
    if key == "your_supabase_key_here":
        print("⚠️  Supabase connection disabled - using placeholder API key")
        print("To enable Supabase: update your API key in data/keys.py or set SUPABASE_KEY environment variable")
        supabase = None
    else:
        supabase: Client = create_client(url, key)
        print("Supabase client created successfully!")
        
except Exception as e:
    print(f"Supabase setup failed: {e}")
    print("Continuing without Supabase connection...")
    supabase = None


Loading ClinicalBERT model...
ClinicalBERT model loaded successfully!
⚠️  Supabase connection disabled - using placeholder API key
To enable Supabase: update your API key in data/keys.py or set SUPABASE_KEY environment variable


In [11]:
# ClinicalBERT model is already initialized in the previous cell
print("ClinicalBERT model is ready for use!")
print(f"Tokenizer vocab size: {tokenizer.vocab_size}")
print(f"Model config: {model.config}")


ClinicalBERT model is ready for use!
Tokenizer vocab size: 119547
Model config: DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "dtype": "float32",
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.56.2",
  "vocab_size": 119547
}



In [12]:
# Python methods which mimic SQL functions (SELECT * FROM PATIENTS)
if supabase is not None:
    try:
        response = supabase.table("Patients").select("*").execute()
        print("Supabase query successful:")
        print(response.data)
    except Exception as e:
        print(f"Supabase query failed: {e}")
else:
    print("Supabase not available - using mock data")
    # Mock data for demonstration
    mock_patients = [
        {"patient_uhid": 1, "patient_name": "Victor Chen", "patient_age": 21},
        {"patient_uhid": 2, "patient_name": "Alice Johnson", "patient_age": 35},
        {"patient_uhid": 3, "patient_name": "Bob Smith", "patient_age": 42}
    ]
    print("Mock patient data:")
    print(mock_patients)

Supabase not available - using mock data
Mock patient data:
[{'patient_uhid': 1, 'patient_name': 'Victor Chen', 'patient_age': 21}, {'patient_uhid': 2, 'patient_name': 'Alice Johnson', 'patient_age': 35}, {'patient_uhid': 3, 'patient_name': 'Bob Smith', 'patient_age': 42}]


In [13]:
# Applying filters to query for a single patient
if supabase is not None:
    try:
        response = supabase.table("Patients").select("*").eq("patient_name", "Victor Chen").execute()
        print("Supabase filtered query successful:")
        print(response.data)
    except Exception as e:
        print(f"Supabase query failed: {e}")
else:
    print("Supabase not available - using mock filtered data")
    # Mock filtered data
    mock_filtered = [{"patient_uhid": 1, "patient_name": "Victor Chen", "patient_age": 21}]
    print("Mock filtered patient data:")
    print(mock_filtered)

Supabase not available - using mock filtered data
Mock filtered patient data:
[{'patient_uhid': 1, 'patient_name': 'Victor Chen', 'patient_age': 21}]


In [14]:
# Chaining responses
if supabase is not None:
    try:
        response = supabase.table("Patients").select("*").execute()
        print("Supabase chained query successful:")
        for row in response.data:
            tests = supabase.table("Tests").select("*").eq("patient_uhid", row['patient_uhid']).execute()
            print(f"Patient {row['patient_uhid']} tests: {tests.data}")
    except Exception as e:
        print(f"Supabase chained query failed: {e}")
else:
    print("Supabase not available - using mock chained data")
    # Mock chained data
    mock_patients = [
        {"patient_uhid": 1, "patient_name": "Victor Chen", "patient_age": 21},
        {"patient_uhid": 2, "patient_name": "Alice Johnson", "patient_age": 35}
    ]
    mock_tests = {
        1: [{"test_id": 1, "patient_uhid": 1, "std_results": True}],
        2: [{"test_id": 2, "patient_uhid": 2, "std_results": False}]
    }
    
    for patient in mock_patients:
        patient_id = patient['patient_uhid']
        tests = mock_tests.get(patient_id, [])
        print(f"Patient {patient_id} tests: {tests}")

Supabase not available - using mock chained data
Patient 1 tests: [{'test_id': 1, 'patient_uhid': 1, 'std_results': True}]
Patient 2 tests: [{'test_id': 2, 'patient_uhid': 2, 'std_results': False}]
